In [ ]:
import datasets

ds_path = "appier-ai-research/StreamBench"
ds_name = "ds_1000"

ds1 = datasets.load_dataset(ds_path, ds_name)

ds_path = "xlangai/DS-1000"
ds2 = datasets.load_dataset(ds_path)

In [ ]:
problematic_ids = [693, 695, 691, 688, 690, 694, 680, 692, 681, 689]

In [ ]:
import re

for row in ds1["test"]:
    if "tensorflow" in row["code_context"]:
        print(len(re.findall("copy.deepcopy", row["code_context"])))

In [ ]:
idx = 379
row = ds1["test"][idx]
print(row["code_context"])
print("=====")
print(row["reference_code"])

# print(row["code_context"].replace("[insert]", row["reference_code"]))

In [ ]:
import tensorflow as tf

In [ ]:
from datasets import load_dataset

dataset_path = "appier-ai-research/robust-finetuning"
dataset_name = "gsm8k"

gsm8k = load_dataset(dataset_path, dataset_name)

In [ ]:
# Load model
import os
from pathlib import Path
from stream_bench.llms.oai_chat import OpenAIChat

os.environ.update({"OAI_KEY": Path("../apikeys/openai_finetune.txt").read_text().strip()})
llm = OpenAIChat(model_name="gpt-4o-mini-2024-07-18")

In [ ]:
for i in range(len(gsm8k["test"]["answer"])):
    a = gsm8k["test"]["answer"][i]
    a = ''.join(a.split("####")[1].strip().split(','))
    if int(a) < 0:
        print(i)

In [ ]:
idx = 1113
q = gsm8k["test"]["question"][idx]
print(q)
print(gsm8k["test"]["answer"][idx])

In [ ]:
print("""\
The following text is an LLM's response to a math question:

Text (enclosed in triple quotes): '''{text}'''

Extract the answer from the text (only extract the digits, potentially the sign if the number is negative), and provide it in the following JSON format:
{{"answer": "<digits>"}}""")

In [ ]:
raw_res, _ = llm(prompt=q)

In [ ]:
print(raw_res)

In [ ]:
prompt_extract = """\
The following text is an LLM's response to a math question:

Text (enclosed in triple quotes): '''{text}'''

Extract the answer from the text (only extract the digits, potentially the sign if the number is negative), and provide it in the following JSON format:
{{"answer": "<digits>"}}"""

prompt = prompt_extract.format(text=raw_res)
res, _ = llm(prompt)

In [ ]:
import json
from stream_bench.benchmarks.utils import extract_json_string

extract_json_string(res)

In [ ]:
from stream_bench.benchmarks.utils import strip_all_lines

def get_prompt() -> str:
    prompt_extract = strip_all_lines("""\
    The following text is an LLM's response to a math question:

    Text (enclosed in triple quotes): '''{text}'''

    Extract the answer from the text (only extract the digits, potentially the sign if the number is negative), and provide it in the following JSON format:
    {{"answer": "<digits>"}}""")
    return prompt_extract.format(text="text")

print(get_prompt())

## MATH

In [ ]:
from datasets import load_dataset

path = "appier-ai-research/robust-finetuning"
name = "math"

math = load_dataset(path, name)

In [ ]:
idx = 0

q = math["test"]["problem"][idx]
a = math["test"]["solution"][idx]

In [ ]:
print(q)
print()
print(a)

In [ ]:
import re

def extract_answer_from_boxed(s: str) -> str:
    """Extract the answer enclosed as follows: boxed{<answer>}"""
    found = re.findall(r"boxed\{(.+)\}", s)
    if len(found) == 1:
        answer = found[0]
    else:
        print(f"Found {len(found)} boxed answers in the string: {s}")
        answer = None
    return answer

for row in math["test"]["solution"]:
    answer = extract_answer_from_boxed(row)
    print(answer)

## Analyze Degeneration

In [ ]:
import json

rows = list()
with open("./log/gsm8k/test/ZeroShotAgent__openai__ft:gpt-4o-mini-2024-07-18:ai-research:gsm8k-100-gt:9u8sLy5N.jsonl") as f:
    for line in f:
        row = json.loads(line)
        if row["num_output_tokens"] == 2048:
            rows.append(row["output_pred"])

rows

In [ ]:
print(rows[0])